In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re
import openpyxl
from datetime  import datetime

In [2]:
#cargar primero el excel o csv
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    nombre_col = ['n_afi', 'parentesco','nombre','dni','no_se','dni2','plan','plan2','sexo','localidad','provincia']
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron_capita= pd.read_csv(ruta_archivo, encoding='ISO-8859-1', sep=';', names=nombre_col)
            #padron_capita = pd.read_excel(ruta_archivo)
            print(padron_capita)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/odontologia (12).csv' mode='r' encoding='cp1252'>
              n_afi  parentesco                          nombre       dni  \
0     1002682185800           0  LUGO RUBEN OMAR                 19780915   
1     1002493263700           0  ARRIOLA DIEGO ALBERTO           19751225   
2     1002213898100           0  TORO PATRICIA DEL CARMEN        19710809   
3     1002155616300           0  VILCA DIONICIO CARLOS           19710508   
4     1002252095300           0  ROCCA CARLOS JAVIER             19730706   
...             ...         ...                             ...       ...   
4245   403970471304           2  GOMEZ BAUTISTA TAHIEL           20180827   
4246   403080024402           2  JUAREZ DANTE BENJAMIN           20150518   
4247   409464120902           2  MENEZES TAIEL EITAN             20080709   
4248   403371628802           2  RODRIGUEZ VICTORIA ABIGAIL      20240524   
4249   403975763202           2  GOMEZ VALENCIA LUCAS

In [3]:
diccionario ={'0':'titular',
              '1':'familiar',
              '2':'familiar',
              '3':'familiar',
              '4':'familiar',
              '5':'familiar',
              '6':'familiar',
              '7':'familiar',
              '8':'familiar'
              }      
#padron_capita['parentesco']=padron_capita['parentesco'].str.replace('\d|\(|\)', '', regex=True)
padron_capita['parentesco']=padron_capita['parentesco'].astype(str).replace(diccionario)

In [4]:
sedes={
'010': 'CABA',
'020': 'ZONA NORTE Y NOROESTE',
'030': 'ZONA OESTE',
'040': 'ZONA SUR',
'050': 'LA PLATA',
'060': 'MAR DEL PLATA',
'070': 'PERGAMINO',
'080': 'SANTA FE',
'090': 'CORDOBA',
'100':'MENDOZA',
'110':'TUCUMAN'
}
padron_capita['n_afi']=padron_capita['n_afi'].astype(str).str.zfill(13)
padron_capita['sede']=padron_capita['n_afi'].str[:3].replace(sedes)



In [21]:
save_path = filedialog.askdirectory()
padron_capita.to_excel(save_path + '\capitasagosto.xlsx', engine='openpyxl', index=False)

In [5]:
print(padron_capita.columns)

Index(['n_afi', 'parentesco', 'nombre', 'dni', 'no_se', 'dni2', 'plan',
       'plan2', 'sexo', 'localidad', 'provincia', 'sede'],
      dtype='object')


### armar la tabla dinamica

In [6]:
df_pivot = padron_capita.pivot_table(index='sede', columns='parentesco', values='dni', aggfunc='count')
df_pivot.loc['Total', :] = df_pivot.sum(axis=0)
df_pivot.loc[:, 'Total'] = df_pivot.sum(axis=1)



df_pivot.columns

Index(['familiar', 'titular', 'Total'], dtype='object', name='parentesco')

In [7]:
fecha = datetime.now()
nombre_mes = fecha.strftime("%B %Y")
save_path = filedialog.askdirectory()
df_pivot.to_excel(save_path + '\\' + nombre_mes + '.xlsx', engine='openpyxl', index=True)